# Otimização de hiperparâmetros da Rede Neural usando o Optuna

## Introdução:

A partir do arquivo [Introdução](Introdução.ipynb) , foi possível identificar parâmetros e características das Features e dos Targets. A partir disso, iremos treinar uma Rede Neural MLP (Multi Layer Perceptron), com o uso do Pytorch, com diversas arquiteturas distintas, através do Optuna, para identificar os melhores hiperparâmetros para a previsão dos nossos Targets.

As Redes Neurais funcionam a partir de Camadas de Neurônios, que recebem uma informação com um determinado peso e processa essa dado através de uma função de ativação e um viés, resultando num output que é passado para outro neurônio conectado ou final. As Redes Neurais possuem numeros variáveis de camadas e neurônios em cada camada. Dessa forma, sendo hiperparâmetros da arquitetura da nossa rede.

Para isso, iremos usar o Optuna, que consegue calcular a métricas de diversas arquiteturas de Redes Neurais. Assim, escolhendo os hiperparâmetros que funcionam melhor para a predição dos nossos targets.

A métrica de erro usada é a Raiz do Erro Quadrático Médio **RMSE**, a qual calcula a raiz da média de diferença entre o valor predito com o real, penalizando grandes diferenças entre os valores (os chamados *outliers*). 

$$RMSE(y,ŷ) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - ŷ_i)^2}
\tag{1}$$

Em que $y$ é o valor real, $ŷ$ é o valor predito e $n$ é o número de exemplos

## Importando os módulos necessários

In [20]:
import os
import tempfile
import pandas as pd
import random as rd
import numpy as np

import optuna
import optuna.visualization as vis
from optuna import load_study

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

### Definindo as constantes do problema:

In [2]:
DEVICE = torch.device("cpu") # Sem GPU
SEMENTE_ALEATORIA = 1249

NUM_TARGETS = 2
NUM_EPOCAS = 50

TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.1

PACIENCIA = 10
MIN_DELTA = 0.001 # Alteração mínima na perda para que se qualifique como melhora

## Pré-processamento:

Carregando o dataset e coletando as colunas que serão usadas como targets e como features:

In [3]:
TARGET = ["Storage_Efficiency_Percentage",
         "GHG_Emission_Reduction_tCO2e"]
         
FEATURES = [
    "Type_of_Renewable_Energy",
    "Grid_Integration_Level",
    "Funding_Sources",
    "Installed_Capacity_MW",
    "Energy_Production_MWh",
    "Energy_Consumption_MWh",
    "Initial_Investment_USD",
    "Air_Pollution_Reduction_Index",
]

In [4]:
df_energia = pd.read_csv('energy_dataset_.csv')
df_energia = df_energia.reindex(FEATURES + TARGET, axis=1)
df_energia = df_energia.dropna()
df_energia

,Type_of_Renewable_Energy,Grid_Integration_Level,Funding_Sources,Installed_Capacity_MW,Energy_Production_MWh,Energy_Consumption_MWh,Initial_Investment_USD,Air_Pollution_Reduction_Index,Storage_Efficiency_Percentage,GHG_Emission_Reduction_tCO2e
0,4,4,1,93.423205,103853.2206,248708.4892,4.732248e+08,81.742461,89.887562,6663.816572
1,4,4,2,590.468942,190223.0649,166104.1642,1.670697e+08,78.139042,84.403343,30656.049820
2,1,2,2,625.951142,266023.4824,424114.6308,8.463610e+07,8.461296,60.498249,1749.613759
3,1,3,2,779.998728,487039.5296,308337.7316,3.967690e+08,8.402441,86.897861,43233.237820
4,3,2,1,242.106837,482815.0856,360437.7705,3.574413e+07,28.822867,70.949351,14858.662760
...,...,...,...,...,...,...,...,...,...,...
14995,3,4,2,745.032555,280007.5738,230544.8268,3.484136e+08,78.923200,90.791405,25234.911810
14996,1,4,3,15.187023,377340.5803,358547.3589,2.560179e+08,54.982974,78.252040,15762.519790
14997,3,1,2,877.539059,480497.3920,214441.6719,1.300112e+08,43.915897,58.282928,44597.809410
14998,7,2,2,551.264716,436383.1694,137043.8713,3.334831e+08,4.877145,73.573666,34363.858000


Dividindo os dados em treino, validação e teste, permitindo que a rede neural se ajuste aos dados de treino e seja otimizada para os dados de validação, evitando o *overfitting*:

In [5]:
indices = df_energia.index
indices_treino_val, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino_val = df_energia.loc[indices_treino_val]
df_teste = df_energia.loc[indices_teste]

X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [6]:
indices = df_treino_val.index
indices_treino, indices_val = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df_energia.loc[indices_treino]
df_val = df_energia.loc[indices_val]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values

X_val = df_val.reindex(FEATURES, axis=1).values
y_val = df_val.reindex(TARGET, axis=1).values

Fazendo uma normalização por Máximo Absoluto, ajustando aos dados de treino e transformando aos demais dados:


In [7]:
scaler = MaxAbsScaler()

# Para os valores de X

scaler.fit(X_treino)

X_treino = scaler.transform(X_treino)
X_teste  = scaler.transform(X_teste)
X_val = scaler.transform(X_val)

# Para os valores de y:

# Organizando em 2 dimensões para o PyTorch
y_treino = y_treino.reshape(-1,2)
y_teste  = y_teste.reshape(-1,2)
y_val = y_val.reshape(-1,2)

# Aplicando a normalização
scaler.fit(y_treino) 

y_treino = scaler.transform(y_treino)
y_teste  = scaler.transform(y_teste)
y_val = scaler.transform(y_val)

Convertendo os dados para Tensores, uma estrutura especial utilizada no módulo `Pytorch`:

In [8]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Agrupando os atributos e *targets* de cada conjunto, utilizando a biblioteca **TensorDataset**. Isso é necessário para o treinamento do modelo, permitindo prever ambos os *targets*, sendo que o processo será feito em **batches**, ou lotes, a fim de otimizar o modelo

In [9]:
treino_dataset = TensorDataset(X_treino, y_treino)
validacao_dataset = TensorDataset(X_val, y_val)
teste_dataset = TensorDataset(X_teste, y_teste)

## Otimização com Optuna:

Para otimizar os hiperparâmetros da nossa rede neural, o Optuna precisa de um modelo que vai criar arquiteturas da MLP, podendo computar métricas com a função objetivo, buscando otimizar essa métrica. A função `define_model` serve para criar uma instância do modelo escolhido, a qual recebe um objeto tipo &ldquo;tentativa&rdquo; do `optuna`. 

### Definindo o modelo:

Os valores dos argumentos podem ser sorteados com as funções `trial.suggest_int` (para números inteiros), `trial.suggest_float` (para números reais) e `trial.suggest_categorical` (para dados categóricos). Assim delimitamos nosso **espaço de busca** dos hiperparâmetros do nosso modelo, em que variamos o número de camadas e de neurônios por camada.

In [10]:
def define_model(trial):
    n_camadas = trial.suggest_int("n_camadas", 2, 5)
    camadas = []

    in_features = len(FEATURES)
    for i in range(n_camadas):
        out_features = trial.suggest_int(f"n_neuronios{i}", 2, 5)
        camadas.append(nn.Linear(in_features, out_features))
        camadas.append(nn.ReLU())
        in_features = out_features

    camadas.append(nn.Linear(in_features, NUM_TARGETS))
    
    return nn.Sequential(*camadas)

### Criando a função objetivo:

A **função objetivo** de um problema de otimização é a função que irá computar a nossa métrica de interesse. Neste caso, a métrica de interesse é a perda de validação do modelo, o qual é treinado ao longo das épocas e sofre Parada Antecipada, visando descobrir uma época adequada em que a perda deixa de diminuir. Aqui variou-se o otimizador usado, a taxa de aprendizado e o tamanho do batch para treinar os Dataloaders, sendo que o método `trial.should_prune()` verifica a perda a cada *trial* e passa pra próxima tentativa caso não haja mudança significativa

In [11]:
def objective(trial):
    #optuna.seed = SEMENTE_ALEATORIA
    torch.manual_seed(SEMENTE_ALEATORIA)
    rd.seed(SEMENTE_ALEATORIA)
    np.random.seed(SEMENTE_ALEATORIA)

    model = define_model(trial).to(DEVICE)    

    nome_otimizador = trial.suggest_categorical("otimizador", ["Adam", "RMSprop", "SGD" ])
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    otimizador = getattr(optim, nome_otimizador)(model.parameters(), lr=lr)

    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    treino_loader = DataLoader(treino_dataset, batch_size=batch_size, shuffle=True)
    validacao_loader = DataLoader(validacao_dataset, batch_size=batch_size, shuffle=False)

    melhor_loss = float("inf")
    min_epoch = 10
    contador_tol = 0

    print(f"Trial {trial.number}")

    for epoch in range(NUM_EPOCAS):        
        #modo de treinamento
        model.train() 

        for batch_data, batch_target in treino_loader:
            batch_data, batch_target = batch_data.to(DEVICE), batch_target.to(DEVICE)
            
            #Foward pass
            output = model(batch_data)

            #Zero Grad
            otimizador.zero_grad()
            
            #Calculo da perda
            loss = F.mse_loss(output, batch_target)

            #Backpropagation
            loss.backward()
            
            #Atualiza os parâmetros
            otimizador.step()

        # modo de validação
        model.eval()
        val_loss_acumulado = 0

        with torch.no_grad():
            for batch_data_val, batch_target_val in validacao_loader: 
                batch_data_val, batch_target_val = batch_data_val.to(DEVICE), batch_target_val.to(DEVICE)
                output_val = model(batch_data_val)
                val_loss_acumulado += F.mse_loss(output_val, batch_target_val, reduction='sum').item() # Soma das perdas

            val_loss = val_loss_acumulado / len(validacao_loader.dataset)
            
        trial.report(val_loss, epoch)  
        
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if epoch >= min_epoch:
            # early stopping
            if val_loss < (melhor_loss - MIN_DELTA):
                melhor_loss = val_loss
                contador_tol = 0
    
            else:
                contador_tol += 1
                if contador_tol >= PACIENCIA:
                    print(f"Parada Antecipada (Early Stopping) na época {epoch}.")
                    val_loss = melhor_loss
                    break

    print(f"[Trial {trial.number} Final] Melhor Val Loss: {melhor_loss:.4f}")
        
    return val_loss

### Realizando o Optuna no HPC:

A otimização em si é realizada criando um objeto de estudo com o `create_study`. O argumento `direction` deve conter a string `"minimize"` pois é um problema de minimização da perda, sendo que ele cria um armazenamento pro estudo

O argumento `study_name` serve para darmos um nome ao processo de busca. O argumento `storage` é utilizado para salvar os resultados da busca em um arquivo de banco de dados. Finalmente, se o argumento `load_if_exists` for `True`, então o `optuna` irá checar se você já realizou um estudo de mesmo nome anteriormente, em caso positivo, o novo estudo será incorporado ao antigo.

In [12]:
NOME_DO_ESTUDO = "Hiperpârametros_Tarrasque_Optuna"

study = optuna.create_study(
    direction="minimize", # analisar se o objetivo é maximizar ou minimizar
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True
)

[I 2025-06-12 18:40:12,414] Using an existing study with name 'Hiperpârametros_Tarrasque_Optuna' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=100)

### Obtendo os melhores parâmetros testados pelo Optuna

In [14]:
melhor_trial = study.best_trial
melhor_batch = study.best_params['batch_size']

print(f"Número do melhor trial: {melhor_trial.number}")
print("Melhor valor:", study.best_value)
print("Melhores hiperparâmetros:")

for key, value in study.best_params.items():
    print(f"  {key}: {value}")

Número do melhor trial: 11
Melhor valor: 0.1045363638136122
Melhores hiperparâmetros:
  n_camadas: 3
  n_neuronios0: 4
  n_neuronios1: 5
  n_neuronios2: 5
  otimizador: Adam
  lr: 0.017405887505001036
  batch_size: 64


### Plotando alguns dados da otimização de hiperparâmetros disponíveis pelo próprio módulo do Optuna

In [15]:
# Histórico da otimização (melhor valor a cada trial)
vis.plot_optimization_history(study).show()

# Importância de cada hiperparâmetro
vis.plot_param_importances(study).show()

## Testando o melhor modelo:

### Carregando o estudo salvo:

In [16]:
objeto_de_estudo_carregado = load_study(
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
)

df_optuna = objeto_de_estudo_carregado.trials_dataframe()

df_optuna

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_n_camadas,params_n_neuronios0,params_n_neuronios1,params_n_neuronios2,params_n_neuronios3,params_n_neuronios4,params_otimizador,state
0,0,0.105557,2025-06-10 17:55:06.048530,2025-06-10 18:11:57.539304,0 days 00:16:51.490774,128,0.000501,2,3,2,NaN,NaN,NaN,SGD,COMPLETE
1,1,0.105106,2025-06-10 18:11:57.613850,2025-06-10 18:26:09.177093,0 days 00:14:11.563243,64,0.001081,5,2,3,2.0,2.0,4.0,SGD,COMPLETE
2,2,0.104617,2025-06-10 18:26:09.264462,2025-06-10 18:26:49.961443,0 days 00:00:40.696981,128,0.002656,2,5,5,NaN,NaN,NaN,RMSprop,COMPLETE
3,3,0.104608,2025-06-10 18:26:50.055524,2025-06-10 18:36:07.877074,0 days 00:09:17.821550,64,0.020148,5,4,5,5.0,4.0,4.0,Adam,COMPLETE
4,4,0.104991,2025-06-10 18:36:07.925366,2025-06-10 18:39:07.748476,0 days 00:02:59.823110,128,0.000465,4,4,2,4.0,5.0,NaN,RMSprop,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,160,0.104637,2025-06-12 18:47:16.692713,2025-06-12 18:47:17.862948,0 days 00:00:01.170235,32,0.025392,4,3,3,4.0,4.0,NaN,Adam,PRUNED
161,161,0.383356,2025-06-12 18:47:17.898754,2025-06-12 18:47:18.580472,0 days 00:00:00.681718,128,0.002667,4,3,3,5.0,4.0,NaN,SGD,PRUNED
162,162,0.108414,2025-06-12 18:47:18.618257,2025-06-12 18:47:19.267697,0 days 00:00:00.649440,64,0.009196,3,3,3,5.0,NaN,NaN,SGD,PRUNED
163,163,0.104592,2025-06-12 18:47:19.305850,2025-06-12 18:47:20.227765,0 days 00:00:00.921915,32,0.012052,4,3,5,3.0,4.0,NaN,SGD,PRUNED


In [21]:
modelo_rede_neural = define_model(melhor_trial)
modelo_rede_neural.eval()

Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=5, bias=True)
  (5): ReLU()
  (6): Linear(in_features=5, out_features=2, bias=True)
)

In [48]:
teste_loader = DataLoader(teste_dataset, batch_size=melhor_batch, shuffle=False)
teste_loss_acumulado = 0

with torch.no_grad():
    for batch_data_teste, batch_target_teste in teste_loader:
        output_teste = modelo(batch_data_teste)
        batch_target_teste = torch.from_numpy(scaler.inverse_transform(batch_target_teste))
        output_teste = torch.from_numpy(scaler.inverse_transform(output_teste))
        teste_loss_acumulado += F.mse_loss(output_teste, batch_target_teste, reduction='sum').item() # Soma das perdas

    teste_loss = teste_loss_acumulado / len(teste_loader.dataset)

In [43]:
modelo_dummy = DummyRegressor()
modelo_dummy.fit(X_teste, y_teste)

y_previsto = modelo_dummy.predict(X_teste)
y_dummy = scaler.inverse_transform(y_previsto)
y_teste = scaler.inverse_transform(y_teste)

In [47]:
rmse_rede_neural = np.sqrt(teste_loss)
rmse_teste = np.sqrt(mean_squared_error(y_teste, y_dummy))

print(f'RMSE da rede neural otimizada: {rmse_rede_neural}\nRMSE do modelo dummy: {rmse_teste}')

RMSE da rede neural otimizada: 15079.886770628742
RMSE do modelo dummy: 10183.716375818405


## Conclusões:

Com base na otimização dos hiperparâmetros da Rede Neural, modificando o otimizador usado, número de camadas e neurônios em cada camada. Obteve-se um conjunto de parâmetros que minimizava a perda do da nossa rede neural.

## Referências:


[1] https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py

[2] https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_checkpoint.py